# Machine Translation

English-German Translation

# 1)- Importing key modules

In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [0]:
# What's life without style :). So, let's add style to our dataframes
#from IPython.core.display import HTML
#css = open('style-table.css').read() + open('style-notebook.css').read()
#HTML('<style>{}</style>'.format(css))

In [0]:
import pandas as pd 
import string 
import pickle
from pickle import dump
from pickle import load
from string import digits
import re 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from numpy import array, argmax, random, take 
import matplotlib.pyplot as plt 
%matplotlib inline 
pd.set_option('display.max_colwidth', 200)

In [4]:
from keras.models import Model
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Embedding,Input,RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint 
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model 
from keras import optimizers 

Using TensorFlow backend.


In [5]:
!  pip install version_information

In [6]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas,re,sklearn, matplotlib,keras

Software versions
Python 3.6.8 64bit [GCC 8.0.1 20180414 (experimental) [trunk revision 259383]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.24.2
re 2.2.1
sklearn 0.21.3
matplotlib 3.0.3
keras 2.2.5
Tue Sep 10 15:40:12 2019 UTC

# 2)- Reading Dataset

In [0]:
lines= pd.read_pickle('full_data.pkl')

In [8]:
lines.shape

(22191, 2)

In [9]:
lines.head()

,eng,ger
0,Prague Stock Market falls to minus by the end of the trading day,Die Prager Börse stürzt gegen Geschäftsschluss ins Minus.
1,After a sharp drop in the morning,Nach dem steilen Abfall am Morgen konnte die Prager Börse die Verluste korrigieren.
2,Transactions with stocks from the Czech Energy Enterprise (ČEZ) reached nearly half of the regular daily trading.,Die Transaktionen mit den Aktien von ČEZ erreichten fast die Hälfte des normalen Tagesgeschäfts.
3,The Prague Stock Market immediately continued its fall from Monday at the beginning of Tuesday's trading,Die Prager Börse knüpfte gleich zu Beginn der Dienstagsgeschäfte an den Einbruch vom Montag an
4,This time the fall in stocks on Wall Street is responsible for the drop.,Diesmal lag der Grund für den Einbruch an der Wall Street.


As this is big data and I have a poor old computing machine. So, I ll use smaller sample. It got to be random to avoid sample biaseness

In [10]:
lines.sample(15)

,eng,ger
20317,How have residents reacted to your terraced house project?,Wie haben die Anwohner auf ihr Reihenhaus-Projekt reagiert?
11873,Kirill Miller is an outstanding man of the St. Petersburg avant-garde of the late 80's early 90's.,Kirill Miller ist eine leuchtende Persönlichkeit der Petersburger Avantgarde Ende der 80er bzw. Anfang der 90er.
4835,"Moreover, Ferrovial, the autonomous government of Castile-La Mancha, and the University of Alcala de Henares signed a protocol of cooperation yesterday to create a Center for the Innovation of Int...","Auf der anderen Seite haben Ferrovial, die Regierung von Kastilien-La Mancha und die Universität von Alcalá de Henares gestern ein Protokoll zur Zusammenarbeit zwecks Schaffung eines Innovationsze..."
8256,The idea of changing cars during races is unusual.,Eine ungewöhnliche Idee ist auch der Wagenwechsel im Laufe der Rennen.
4714,"The intervention of the aid, however, calls for the full braking system, reducing the distance the car needs to stop.","Die Bremshilfe dagegen greift mit maximaler Kraft in das Bremssystem ein und verringert dadurch die Bremszeit und somit auch den Bremsabstand, den ein Auto beim Bremsvorgang zurücklegt."
20080,A couple of half-centuries and Lyth might have seen his run extended but he was in control of his own destiny.,"Vor langer Zeit hätte man den Lauf von Lyth verlängert, aber er hatte die Kontrolle über sein eigenes Schicksal."
14993,"In their first three appearances in the European top flight, the Munich team put in a good performance for long periods, but in the crucial phases were not sufficiently focussed.","Bei ihrem erst dritten Auftritt in der europäischen Königsklasse verkauften sich die Münchner lange sehr gut, waren in den entscheidenden Phasen aber einfach nicht abgeklärt genug."
4022,"As soon as Frankfurt is blanketed in snow and the temperatures drop below zero, 20 so-called ""A Routes"" are serviced by large-scale distribution vehicles, explains Brauburger.","Sobald es in Frankfurt flächendeckend schneit und die Temperaturen unter Null Grad liegen, werden 20 sogenannte A-Strecken mit Großstreufahrzeugen abgefahren, erläutert Brauburger."
9865,"""She said in an aggressive tone, ""You shouldn't have left your daughter by the door,"""" Roseanna Monk said.","""Sie sagte in einem aggressiven Ton: ""Sie sollten Ihre Tochter nicht in der Nähe der Tür absetzen"", sagte Roseanna Monk."
18060,"The reason that we are feeling its deadly effects now is that, though asbestos use has been illegal for years (all types of asbestos were eventually banned by law in 1999), it usually takes decade...","Der Grund, warum wir die tödlichen Auswirkungen jetzt erleben, obwohl Asbest seit Jahren illegal ist (1999 wurden alle Asbesttypen verboten), ist der, dass ein Mesotheliom sich erst nach Jahrzehnt..."


In [0]:
lines = lines[:5000]

# 3)- Quick Preprocessing

In [0]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.deu=lines.ger.apply(lambda x: x.lower())

In [0]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.deu=lines.ger.apply(lambda x: re.sub("'", '', x))

In [0]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.deu=lines.ger.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.deu=lines.ger.apply(lambda x: x.translate(remove_digits))

In [0]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.deu=lines.ger.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.deu=lines.ger.apply(lambda x: re.sub(" +", " ", x))

In [16]:
lines.sample(10)

,eng,ger
2882,more links to the capital will be available for example to the people in the ostrava region as the railways are adding one pendolino to that line,"Mehrere Verbindungsmöglichkeiten in die Hauptstadt erhalten zum Beispiel die Bewohner des Ostrava-Gebiets, die Eisenbahn setzt für diese Strecke Züge des Typs Pendolino ein."
3198,the new us policy stresses that if the cuban government takes concrete steps such as freeing political prisoners and creating more space for opposition the united states will reciprocate,"Die neue US-Politik legt Wert darauf, dass die Vereinigten Staaten eine Gegenleistung erbringen werden, wenn die kubanische Regierung konkrete Schritte in diese Richtung unternimmt, wie Freilassun..."
4006,around a hundred people also demonstrated in istanbul,Hunderte Menschen demonstrierten ebenfalls in Istanbul.
4423,whether these illusions are perceived to be pleasant or unpleasant is often only a matter of perspective,"Ob Trugbilder als angenehm oder unangenehm empfunden werden, ist oft nur eine Frage des Standpunkts."
261,if the mobile operators are looking the other way concerning this procedure,Verschließen die Mobilfunkanbieter vor einem solchen Vorgehen die Augen
480,they travel at night,Nachts hin
886,at the time,Zu diesem Zeitpunkt argumentierte das Ministerium
4982,they work with users in garages technicians and insurance companies,"Sie arbeiten mit 24.000 Benutzern in Werkstätten, 3.100 Gutachtern und 53 Versicherungsgesellschaften zusammen."
3607,if you add up all the placement work proposed before the exam you already end up with more presence in front of students which equates to over hours he continued,"Er sagte weiterhin : ""Wenn Sie alle vor der Leistungsprüfung angebotenen Praktika hinzufügen, erreichen Sie noch mehr Stunden, die Sie vor den Schülern verbracht haben, d.h. mehr als 300 Stunden."""
1700,there is a border along which there has been no agreement for too long a time and,Es gibt eine Grenzlinie


In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
# Vocabulary of German 
all_german_words=set()
for ger in lines.ger:
    for word in ger.split():
        if word not in all_german_words:
            all_german_words.add(word)

In [18]:
# Max Length of source sequence
import numpy as np
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

74

In [19]:
# Max Length of target sequence
lenght_list=[]
for l in lines.ger:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

86

# Make a threshold here

### 3a)- Defining input and target

In [20]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_german_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_german_words)
num_encoder_tokens, num_decoder_tokens

(10955, 20535)

In [21]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

20536

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

### 3b)-Train - Test Split

For validation

In [24]:
X, y = lines.eng, lines.ger #X being input, y being target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape, X_test.shape

((4000,), (1000,))

**Save the train and test dataframes for reproducing the results later, as they are shuffled**

In [0]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# 4)-Encoder - Decoder Model Architecture

In [0]:
latent_dim = 50

In [28]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [30]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
#from IPython.display import Image
#Image(retina=True, filename='train.png')

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 15

In [33]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     547750      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     1026800     input_2[0][0]                    
____________________________________________________________________________________________

In [34]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)


Epoch 1/15
31/31 [==============================] - 35s 1s/step - loss: 9.2373 - acc: 0.0284 - val_loss: 8.4184 - val_acc: 0.0392
Epoch 2/15
31/31 [==============================] - 30s 969ms/step - loss: 8.0842 - acc: 0.0366 - val_loss: 8.2181 - val_acc: 0.0381
Epoch 3/15
31/31 [==============================] - 30s 966ms/step - loss: 7.9106 - acc: 0.0366 - val_loss: 8.3575 - val_acc: 0.0379
Epoch 4/15
31/31 [==============================] - 30s 969ms/step - loss: 7.8832 - acc: 0.0359 - val_loss: 8.5018 - val_acc: 0.0387
Epoch 5/15
31/31 [==============================] - 30s 960ms/step - loss: 7.8660 - acc: 0.0362 - val_loss: 8.6565 - val_acc: 0.0372
Epoch 6/15
31/31 [==============================] - 30s 963ms/step - loss: 7.8450 - acc: 0.0358 - val_loss: 8.7571 - val_acc: 0.0385
Epoch 7/15
31/31 [==============================] - 30s 959ms/step - loss: 7.8127 - acc: 0.0363 - val_loss: 8.8164 - val_acc: 0.0373
Epoch 8/15
31/31 [==============================] - 30s 954ms/step - lo

In [0]:
# save model
model.save_weights('translate.h5')

In [0]:
model.load_weights('translate.h5')

# Inference Setup

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Decode sample sequeces

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    """
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']
    
    """

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Evaluation on Train Dataset

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [40]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual German Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: the hang seng index of the hong kong stock exchange wrote off nearly four percent during the day
Actual German Translation: dex der Hang Seng Börse in Hongkong verlor im Laufe des Geschäftstages fast vier Prozentpu
Predicted German Translation:  der ist der ist der der ist der der der der der


In [41]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual German Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: the mayor of gioia tauro was disposed towards conforming municipal interests with those of the piromalli gang
Actual German Translation: ürgermeister von Gioia Tauro war be
Predicted German Translation:  der ist der ist der der ist der der der der der


In [42]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual German Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: i shot the photo of this post during the microsoft employees break
Actual German Translation: to von diesem Ort habe ich in Red
Predicted German Translation:  der ist der der ist der der der ist der der der


# Evaluation on Validation Dataset

In [0]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [44]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual German Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: we have made great progress towards an agreement that will be effective on the market
Actual German Translation: aben große Fortschritte in Richtung einer Einigung gem
Predicted German Translation:  der ist der der ist der der der ist der der der


In [45]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual German Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: however
Actual German Translation: mt jedoch darau
Predicted German Translation:  der der ist der der der der der der der der der


In [46]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual German Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: the appointment of a special prosecutor with the power to force witnesses to testify was one of his key recommendations
Actual German Translation: rufung eines Spezialermittlers mit der Befu
Predicted German Translation:  der ist der der ist der der der ist der der der
